# Framing

In questa fase bisogna considerare il video e il file contenente tutti i joint della mano.

La prima cosa da fare è estrarre la durata in millisecondi dell'intero video. Per fare questo è stata creata una classe di supporto chiamata ExtractVideoDuration.

In [1]:
from modules.select_file import SelectFile
from modules.extract_video_duration import ExtractVideoDuration

In [2]:
video_duration = ExtractVideoDuration.get_duration(SelectFile.get_path("Insert the absolute path of the video file:"))

Insert the absolute path of the video file:
/home/gigi-g/OneDrive/Tesi1/2021-04-07-00-48-37-381/2021-04-07-00-48-37-381_Video.mp4


In [3]:
video_duration

105970

Una volta ottenuto il tempo, quest'ultimo deve essere suddiviso in intervalli da 33 millisecondi.

In [4]:
video_duration/(1000/30)

3179.1

In [5]:
frame_rate = 30

In [6]:
win_size = 1000/frame_rate

Come possiamo osservare in totale andremo a considerare 3211 intervalli, per ognuno di questi intervalli bisogna considerare solo l'azione più recente, quindi quella con un timestamp maggiore.

Adesso consideriamo il file contenente tutti i joint della mano. Per estrarre queste informazioni è stata realizzata la classe HandPose, inoltre questa classe semplificherà l'utilizzo di queste informazioni.

In [7]:
from io import FileIO

class HandPose:

    _hand_pose:list = []
    _start_time:int = 0

    def __init__(self, path:str) -> None:
        try:
            self.__extract_hand_pose(open(path, "r"))
        except ValueError:
            print("ERROR: file not found.")
            exit(-1)
    
    def _get_millis(self, timestamp:str) -> int:
        hours, minutes, seconds, millis = [(int(elem)) for elem in timestamp.split("-")]
        return (hours * 3600 + minutes * 60 + seconds) * 1000 + millis
    
    def __extract_hand_pose(self, file_pose:FileIO) -> None:
        self._start_time = self._get_millis(file_pose.readline())
        self._hand_pose = [(pose) for pose in file_pose]
    
    def print_hand_pose(self, num:int) -> None:
        index:int = 0
        while index < num:
            print(self._hand_pose[num])
            index += 1
            
    def get_start_time(self) -> int:
        return self._start_time
    
    def get_hand_pose_dict(self) -> dict:
        d = {}
        for pose in self._hand_pose:
            timestamp, hand, other = pose.replace(",", ".").split()
            key:str = str(self._get_millis(timestamp))
            joint_info:list = other.split("_")
            if key not in d:
                d[key] = {
                    hand + "_" + joint_info[0] + "_x": float(joint_info[2]),
                    hand + "_" + joint_info[0] + "_y": float(joint_info[4]),
                    hand + "_" + joint_info[0] + "_z": float(joint_info[6])
                }
            else:
                d[key][hand + "_" + joint_info[0] + "_x"] = float(joint_info[2])
                d[key][hand + "_" + joint_info[0] + "_y"] = float(joint_info[4])
                d[key][hand + "_" + joint_info[0] + "_z"] = float(joint_info[6])
        return d

In [8]:
hand_pose:HandPose = HandPose(SelectFile.get_path("Insert the absolute path of the hand pose file:"))

Insert the absolute path of the hand pose file:
/home/gigi-g/OneDrive/Tesi1/2021-04-07-00-48-37-381/2021-04-07-00-48-37-381_handPose3D.txt


In [9]:
hand_pose.print_hand_pose(5)

00-48-41-821 LEFT IndexTip_x_-0,08355514_y_0,08958711_z_0,5297722

00-48-41-821 LEFT IndexTip_x_-0,08355514_y_0,08958711_z_0,5297722

00-48-41-821 LEFT IndexTip_x_-0,08355514_y_0,08958711_z_0,5297722

00-48-41-821 LEFT IndexTip_x_-0,08355514_y_0,08958711_z_0,5297722

00-48-41-821 LEFT IndexTip_x_-0,08355514_y_0,08958711_z_0,5297722



In [10]:
hand_pose.get_start_time()

2921816

Ogni riga del file contiene le segueni informazioni:

- Timestamp;
- Mano;
- Il tipo di joint;
- Valore sull'asse x;
- Valore sull'asse y;
- Valore sull'asse z.

Quello che si vuole fare è ottenere un dataframe che ha nelle colonne le informazioni sopra elencate.

In [11]:
hand_pose_dict = hand_pose.get_hand_pose_dict()
hand_pose_dict

{'2921821': {'RIGHT_Palm_x': 0.1361058,
  'RIGHT_Palm_y': -0.02834186,
  'RIGHT_Palm_z': 0.5042763,
  'LEFT_Palm_x': -0.1436025,
  'LEFT_Palm_y': -0.01306666,
  'LEFT_Palm_z': 0.494088,
  'RIGHT_ThumbTip_x': 0.04105591,
  'RIGHT_ThumbTip_y': 0.007040426,
  'RIGHT_ThumbTip_z': 0.5222733,
  'LEFT_ThumbTip_x': -0.04144681,
  'LEFT_ThumbTip_y': 0.007697612,
  'LEFT_ThumbTip_z': 0.5107116,
  'RIGHT_IndexTip_x': 0.08965623,
  'RIGHT_IndexTip_y': 0.08499929,
  'RIGHT_IndexTip_z': 0.5274367,
  'LEFT_IndexTip_x': -0.08355514,
  'LEFT_IndexTip_y': 0.08958711,
  'LEFT_IndexTip_z': 0.5297722,
  'RIGHT_MiddleTip_x': 0.1367853,
  'RIGHT_MiddleTip_y': 0.09367665,
  'RIGHT_MiddleTip_z': 0.5396325,
  'LEFT_MiddleTip_x': -0.1230137,
  'LEFT_MiddleTip_y': 0.1078383,
  'LEFT_MiddleTip_z': 0.5287636,
  'RIGHT_RingTip_x': 0.1658046,
  'RIGHT_RingTip_y': 0.07832579,
  'RIGHT_RingTip_z': 0.5368138,
  'LEFT_RingTip_x': -0.1525814,
  'LEFT_RingTip_y': 0.09794147,
  'LEFT_RingTip_z': 0.5232129,
  'RIGHT_PinkyTip

Attraverso la funzione get_hand_pose_dict è stato ricavato un dizionario all'interno del quale sono presenti tutti i joint catturati in un istante di tempo.

In [12]:
hand_joints:list = []
for itm in hand_pose_dict['2921821']:
    hand_joints.append(itm)
print("data = {")
for x in hand_joints:
    print("\t\"" + x + "\": [],")
print("}")

data = {
	"RIGHT_Palm_x": [],
	"RIGHT_Palm_y": [],
	"RIGHT_Palm_z": [],
	"LEFT_Palm_x": [],
	"LEFT_Palm_y": [],
	"LEFT_Palm_z": [],
	"RIGHT_ThumbTip_x": [],
	"RIGHT_ThumbTip_y": [],
	"RIGHT_ThumbTip_z": [],
	"LEFT_ThumbTip_x": [],
	"LEFT_ThumbTip_y": [],
	"LEFT_ThumbTip_z": [],
	"RIGHT_IndexTip_x": [],
	"RIGHT_IndexTip_y": [],
	"RIGHT_IndexTip_z": [],
	"LEFT_IndexTip_x": [],
	"LEFT_IndexTip_y": [],
	"LEFT_IndexTip_z": [],
	"RIGHT_MiddleTip_x": [],
	"RIGHT_MiddleTip_y": [],
	"RIGHT_MiddleTip_z": [],
	"LEFT_MiddleTip_x": [],
	"LEFT_MiddleTip_y": [],
	"LEFT_MiddleTip_z": [],
	"RIGHT_RingTip_x": [],
	"RIGHT_RingTip_y": [],
	"RIGHT_RingTip_z": [],
	"LEFT_RingTip_x": [],
	"LEFT_RingTip_y": [],
	"LEFT_RingTip_z": [],
	"RIGHT_PinkyTip_x": [],
	"RIGHT_PinkyTip_y": [],
	"RIGHT_PinkyTip_z": [],
	"LEFT_PinkyTip_x": [],
	"LEFT_PinkyTip_y": [],
	"LEFT_PinkyTip_z": [],
	"RIGHT_ThumbDistalJoint_x": [],
	"RIGHT_ThumbDistalJoint_y": [],
	"RIGHT_ThumbDistalJoint_z": [],
	"LEFT_ThumbDistalJoint_x": [],
	"

In [13]:
data = {
	"RIGHT_Palm_x": [],
	"RIGHT_Palm_y": [],
	"RIGHT_Palm_z": [],
	"LEFT_Palm_x": [],
	"LEFT_Palm_y": [],
	"LEFT_Palm_z": [],
	"RIGHT_ThumbTip_x": [],
	"RIGHT_ThumbTip_y": [],
	"RIGHT_ThumbTip_z": [],
	"LEFT_ThumbTip_x": [],
	"LEFT_ThumbTip_y": [],
	"LEFT_ThumbTip_z": [],
	"RIGHT_IndexTip_x": [],
	"RIGHT_IndexTip_y": [],
	"RIGHT_IndexTip_z": [],
	"LEFT_IndexTip_x": [],
	"LEFT_IndexTip_y": [],
	"LEFT_IndexTip_z": [],
	"RIGHT_MiddleTip_x": [],
	"RIGHT_MiddleTip_y": [],
	"RIGHT_MiddleTip_z": [],
	"LEFT_MiddleTip_x": [],
	"LEFT_MiddleTip_y": [],
	"LEFT_MiddleTip_z": [],
	"RIGHT_RingTip_x": [],
	"RIGHT_RingTip_y": [],
	"RIGHT_RingTip_z": [],
	"LEFT_RingTip_x": [],
	"LEFT_RingTip_y": [],
	"LEFT_RingTip_z": [],
	"RIGHT_PinkyTip_x": [],
	"RIGHT_PinkyTip_y": [],
	"RIGHT_PinkyTip_z": [],
	"LEFT_PinkyTip_x": [],
	"LEFT_PinkyTip_y": [],
	"LEFT_PinkyTip_z": [],
	"RIGHT_ThumbDistalJoint_x": [],
	"RIGHT_ThumbDistalJoint_y": [],
	"RIGHT_ThumbDistalJoint_z": [],
	"LEFT_ThumbDistalJoint_x": [],
	"LEFT_ThumbDistalJoint_y": [],
	"LEFT_ThumbDistalJoint_z": [],
	"RIGHT_IndexDistalJoint_x": [],
	"RIGHT_IndexDistalJoint_y": [],
	"RIGHT_IndexDistalJoint_z": [],
	"LEFT_IndexDistalJoint_x": [],
	"LEFT_IndexDistalJoint_y": [],
	"LEFT_IndexDistalJoint_z": [],
	"RIGHT_MiddleDistalJoint_x": [],
	"RIGHT_MiddleDistalJoint_y": [],
	"RIGHT_MiddleDistalJoint_z": [],
	"LEFT_MiddleDistalJoint_x": [],
	"LEFT_MiddleDistalJoint_y": [],
	"LEFT_MiddleDistalJoint_z": [],
	"RIGHT_RingDistalJoint_x": [],
	"RIGHT_RingDistalJoint_y": [],
	"RIGHT_RingDistalJoint_z": [],
	"LEFT_RingDistalJoint_x": [],
	"LEFT_RingDistalJoint_y": [],
	"LEFT_RingDistalJoint_z": [],
	"RIGHT_PinkyDistalJoint_x": [],
	"RIGHT_PinkyDistalJoint_y": [],
	"RIGHT_PinkyDistalJoint_z": [],
	"LEFT_PinkyDistalJoint_x": [],
	"LEFT_PinkyDistalJoint_y": [],
	"LEFT_PinkyDistalJoint_z": [],
	"RIGHT_ThumbProximalJoint_x": [],
	"RIGHT_ThumbProximalJoint_y": [],
	"RIGHT_ThumbProximalJoint_z": [],
	"LEFT_ThumbProximalJoint_x": [],
	"LEFT_ThumbProximalJoint_y": [],
	"LEFT_ThumbProximalJoint_z": [],
	"RIGHT_IndexKnuckle_x": [],
	"RIGHT_IndexKnuckle_y": [],
	"RIGHT_IndexKnuckle_z": [],
	"LEFT_IndexKnuckle_x": [],
	"LEFT_IndexKnuckle_y": [],
	"LEFT_IndexKnuckle_z": [],
	"RIGHT_MiddleKnuckle_x": [],
	"RIGHT_MiddleKnuckle_y": [],
	"RIGHT_MiddleKnuckle_z": [],
	"LEFT_MiddleKnuckle_x": [],
	"LEFT_MiddleKnuckle_y": [],
	"LEFT_MiddleKnuckle_z": [],
	"RIGHT_RingKnuckle_x": [],
	"RIGHT_RingKnuckle_y": [],
	"RIGHT_RingKnuckle_z": [],
	"LEFT_RingKnuckle_x": [],
	"LEFT_RingKnuckle_y": [],
	"LEFT_RingKnuckle_z": [],
	"RIGHT_PinkyKnuckle_x": [],
	"RIGHT_PinkyKnuckle_y": [],
	"RIGHT_PinkyKnuckle_z": [],
	"LEFT_PinkyKnuckle_x": [],
	"LEFT_PinkyKnuckle_y": [],
	"LEFT_PinkyKnuckle_z": [],
	"RIGHT_IndexMiddleJoint_x": [],
	"RIGHT_IndexMiddleJoint_y": [],
	"RIGHT_IndexMiddleJoint_z": [],
	"LEFT_IndexMiddleJoint_x": [],
	"LEFT_IndexMiddleJoint_y": [],
	"LEFT_IndexMiddleJoint_z": [],
	"RIGHT_MiddleMiddleJoint_x": [],
	"RIGHT_MiddleMiddleJoint_y": [],
	"RIGHT_MiddleMiddleJoint_z": [],
	"LEFT_MiddleMiddleJoint_x": [],
	"LEFT_MiddleMiddleJoint_y": [],
	"LEFT_MiddleMiddleJoint_z": [],
	"RIGHT_RingMiddleJoint_x": [],
	"RIGHT_RingMiddleJoint_y": [],
	"RIGHT_RingMiddleJoint_z": [],
	"LEFT_RingMiddleJoint_x": [],
	"LEFT_RingMiddleJoint_y": [],
	"LEFT_RingMiddleJoint_z": [],
	"RIGHT_PinkyMiddleJoint_x": [],
	"RIGHT_PinkyMiddleJoint_y": [],
	"RIGHT_PinkyMiddleJoint_z": [],
	"LEFT_PinkyMiddleJoint_x": [],
	"LEFT_PinkyMiddleJoint_y": [],
	"LEFT_PinkyMiddleJoint_z": [],
	"RIGHT_ThumbMetacarpalJoint_x": [],
	"RIGHT_ThumbMetacarpalJoint_y": [],
	"RIGHT_ThumbMetacarpalJoint_z": [],
    "LEFT_ThumbMetacarpalJoint_x": [],
	"LEFT_ThumbMetacarpalJoint_y": [],
	"LEFT_ThumbMetacarpalJoint_z": [],
    'RIGHT_IndexMetacarpal_x': [],
    'RIGHT_IndexMetacarpal_y': [],
    'RIGHT_IndexMetacarpal_z': [],
    'LEFT_IndexMetacarpal_x': [],
    'LEFT_IndexMetacarpal_y': [],
    'LEFT_IndexMetacarpal_z': [],
    'RIGHT_MiddleMetacarpal_x': [],
    'RIGHT_MiddleMetacarpal_y': [],
    'RIGHT_MiddleMetacarpal_z': [],
    'LEFT_MiddleMetacarpal_x': [],
    'LEFT_MiddleMetacarpal_y': [],
    'LEFT_MiddleMetacarpal_z': [],
    'RIGHT_RingMetacarpal_x': [],
    'RIGHT_RingMetacarpal_y': [],
    'RIGHT_RingMetacarpal_z': [],
    'LEFT_RingMetacarpal_x': [],
    'LEFT_RingMetacarpal_y': [],
    'LEFT_RingMetacarpal_z': [],
    'RIGHT_PinkyMetacarpal_x': [],
    'RIGHT_PinkyMetacarpal_y': [],
    'RIGHT_PinkyMetacarpal_z': [],
    'LEFT_PinkyMetacarpal_x': [],
    'LEFT_PinkyMetacarpal_y': [],
    'LEFT_PinkyMetacarpal_z': [],
    'RIGHT_Wrist_x': [],
    'RIGHT_Wrist_y': [],
    'RIGHT_Wrist_z': [],
    'LEFT_Wrist_x': [],
    'LEFT_Wrist_y': [],
    'LEFT_Wrist_z': [],
}

In [14]:
hand_joints = []
for key in data:
    hand_joints.append(key)

In [15]:
tim:int = video_duration
start_tim:int = hand_pose.get_start_time()
index_data:int = 0
while tim > 0:
    index:int = 0
    for key in data:
        data[key].append(0)
    tim -= win_size
    if tim >= 0:
        while index < win_size:
            if str(start_tim) in hand_pose_dict:
                for key, val in hand_pose_dict[str(start_tim)].items():
                    data[key][index_data] = val
            start_tim += 1
            index += 1
    else:
        while index < (win_size + tim):
            if str(start_tim) in hand_pose_dict:
                for key, val in hand_pose_dict[str(start_tim)].items():
                    data[key][index_data] = val
            start_tim += 1
            index += 1
    index_data += 1

In [16]:
print(video_duration + hand_pose.get_start_time())

3027786


In [17]:
for key in data:
    print(data[key])
    break

[0.1361058, 0.1345574, 0.135006, 0.1369882, 0.1363625, 0.1368733, 0, 0.1373993, 0.1376307, 0.1369988, 0.136657, 0.1366565, 0.1359425, 0.1359488, 0, 0.136315, 0.1365353, 0.1361962, 0.1360883, 0.1371056, 0, 0.1361884, 0.1366339, 0.1372083, 0.1388856, 0.1376697, 0.1367823, 0.1371211, 0.1379102, 0.1374102, 0.1374139, 0.1373982, 0.1372504, 0.1364916, 0.1351374, 0.134891, 0.135644, 0.1365506, 0.1360021, 0.1351867, 0.1344586, 0.1344082, 0.1341088, 0.1338976, 0, 0.1339612, 0.1336553, 0.1322233, 0.1304784, 0.1301984, 0.1296417, 0.1297902, 0, 0.1287302, 0, 0.1281013, 0.1275763, 0.1264569, 0.1260099, 0, 0.1283226, 0, 0.1278706, 0.1284066, 0.1279805, 0.1272459, 0.1274503, 0.1274127, 0, 0.1255598, 0.1260059, 0.1250437, 0.1239219, 0.1233073, 0.1224708, 0.121025, 0.1203811, 0.1215293, 0.121478, 0.1208508, 0.1180536, 0.1169973, 0.1168254, 0.1164289, 0.1158746, 0.1153386, 0.1153927, 0.1161525, 0.1159737, 0.1159735, 0.1161316, 0.1164184, 0.1160404, 0.1155924, 0.1149909, 0.1155294, 0.1164466, 0.1172865, 

In [18]:
import pandas as pd

In [19]:
df = pd.DataFrame(data,columns=hand_joints)

In [20]:
df

,RIGHT_Palm_x,RIGHT_Palm_y,RIGHT_Palm_z,LEFT_Palm_x,LEFT_Palm_y,LEFT_Palm_z,RIGHT_ThumbTip_x,RIGHT_ThumbTip_y,RIGHT_ThumbTip_z,LEFT_ThumbTip_x,...,RIGHT_PinkyMetacarpal_z,LEFT_PinkyMetacarpal_x,LEFT_PinkyMetacarpal_y,LEFT_PinkyMetacarpal_z,RIGHT_Wrist_x,RIGHT_Wrist_y,RIGHT_Wrist_z,LEFT_Wrist_x,LEFT_Wrist_y,LEFT_Wrist_z
0,0.136106,-0.028342,0.504276,-0.143602,-0.013067,0.494088,0.041056,0.007040,0.522273,-0.041447,...,0.493076,-0.158524,-0.043128,0.481324,0.127647,-0.070477,0.479784,-0.140429,-0.055822,0.469097
1,0.134557,-0.027701,0.504807,-0.144994,-0.012649,0.493745,0.039396,0.007685,0.522272,-0.042953,...,0.493685,-0.159840,-0.042732,0.480951,0.126239,-0.069853,0.480289,-0.141675,-0.055392,0.468809
2,0.135006,-0.029101,0.504659,-0.144397,-0.013471,0.493747,0.039947,0.006667,0.522045,-0.042289,...,0.493606,-0.159308,-0.043492,0.480892,0.126522,-0.071243,0.480231,-0.141182,-0.056131,0.468678
3,0.136988,-0.028733,0.503911,-0.142682,-0.012712,0.494288,0.042039,0.007355,0.521290,-0.040461,...,0.493217,-0.158012,-0.042791,0.481284,0.127903,-0.070923,0.479887,-0.139945,-0.055469,0.469017
4,0.136362,-0.028703,0.503735,-0.143380,-0.012937,0.493749,0.041309,0.007276,0.520998,-0.041185,...,0.493013,-0.157719,-0.042696,0.481494,0.128316,-0.070940,0.479696,-0.139668,-0.055395,0.469222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3175,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3176,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3177,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3178,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
